In [31]:
import os
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings
from dotenv import load_dotenv
from langchain_community.document_loaders  import CSVLoader
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.prompts import PromptTemplate

In [2]:
load_dotenv()
nvidia_api_key = os.getenv("NVIDIA_API_KEY")

In [3]:
loader = CSVLoader(file_path='D:/corp8.ai internship/recipe_cleaned.csv', encoding='ISO-8859-1')
docs = loader.load()

In [4]:
final_docs = []
for i in range(0,999):
    final_docs.append(docs[i].page_content)

In [5]:
final_docs

['title: Chicory Salad With Creamy Garlic Dressing\ndirections: ["Place bread cubes in a bowl; drizzle oil over the bread cubes, tossing well. Arrange bread cubes in single layer on a baking sheet. Bake at 400\\u00b0 for 8 minutes or until golden. Let cool.", "Place cottage cheese in food processor, and process 2 minutes or until smooth, scraping sides of bowl once. Add the pepper and next 4 ingredients; process until well-blended. Spoon mixture into a bowl; set aside.", "Combine the chicory, romaine, and bread cubes in a bowl; toss gently. Add dressing, toss gently to coat. Top with chopped egg."]\nIngridents: ["red wine vinegar", "torn romaine lettuce", "pepper", "eggs", "cheese", "italian bread", "garlic", "mustard", "olive oil", "salt"]\nlink: www.myrecipes.com/recipe/chicory-salad-with-creamy-garlic-dressing',
 'title: Basta\ndirections: ["Mill the peanuts.", "Melt butter and sugar, and mix in peanuts.", "Brush 1 sheet phyllo with a little melted butter.", "Add small blobs of pean

In [7]:
len(final_docs)

999

In [84]:
final_docs = str(final_docs)
final_docs
type(final_docs)


str

In [ ]:
length_function = len

# The default list of split characters is [\n\n, \n, " ", ""]
# Tries to split on them in order until the chunks are small enough
# Keep paragraphs, sentences, words together as long as possible
splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", " ", ""],
    chunk_size=600, 
    chunk_overlap=120,
    length_function=length_function,
)
text = final_docs
splits = splitter.split_text(text)

In [18]:
splits

['[\'title: Chicory Salad With Creamy Garlic Dressing\\ndirections: ["Place bread cubes in a bowl; drizzle oil over the bread cubes, tossing well. Arrange bread cubes in single layer on a baking sheet. Bake at 400\\\\u00b0 for 8 minutes or until golden. Let cool.", "Place cottage cheese in food processor, and process 2 minutes or until smooth, scraping sides of bowl once. Add the pepper and next 4 ingredients; process until well-blended. Spoon mixture into a bowl; set aside.", "Combine the chicory, romaine, and bread cubes in a bowl; toss gently. Add dressing, toss gently to coat. Top with chopped',
 'the chicory, romaine, and bread cubes in a bowl; toss gently. Add dressing, toss gently to coat. Top with chopped egg."]\\nIngridents: ["red wine vinegar", "torn romaine lettuce", "pepper", "eggs", "cheese", "italian bread", "garlic", "mustard", "olive oil", "salt"]\\nlink: www.myrecipes.com/recipe/chicory-salad-with-creamy-garlic-dressing\', \'title: Basta\\ndirections: ["Mill the peanut

In [20]:
len(splits)

1595

In [21]:
embeddings = NVIDIAEmbeddings(
    model="nvidia/nv-embedqa-e5-v5",
    model_type="passage",
    api_key=nvidia_api_key
)

vector_store = Chroma.from_texts(
    embedding = embeddings,
    texts = splits,
    persist_directory='my_chroma_db',
    collection_name='sample'
)

In [24]:
vector_store.similarity_search(
    query='ingredients :"bite size shredded rice biscuits", "vanilla", "brown sugar", "nuts", "milk", "butter"',
    k=5
)


[Document(metadata={}, page_content='"Keep airtight until you wish to serve these.", "Otherwise, enjoy munching on these side-by-side with a cup of hot tea!"]\\nIngridents: ["condensed milk", "bread", "coconut", "butter"]\\nlink: www.food.com/recipe/tried-and-true-recipe-for-crispy-and-tasty-snack-coconut-fingers-47569\', \'title: Chocolate Chip Kiss Cookies\\ndirections: ["Remove foil from Kisses.", "In a large bowl, combine butter, sugars and vanilla. Beat well. Add flour; blend until smooth. Stir in chips. Mold scant tablespoon of dough around each Kiss, covering completely.", "Shape into ball and place on ungreased cookie sheet.'),
 Document(metadata={}, page_content='"eggs", "lemon extract", "pecans", "raisins", "flour", "mayonnaise", "candied pineapple", "candied cherries"]\\nlink: www.cookbooks.com/Recipe-Details.aspx?id=237790\', \'title: Rice Pudding\\ndirections: ["Scald milk.", "Add sugar, salt and margarine.", "Stir until margarine is melted.", "Slowly add milk mixture to e

In [74]:
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 30})

In [75]:
retriever.invoke('ingredients :"bite size shredded rice biscuits", "vanilla", "brown sugar", "nuts", "milk", "butter"')

[Document(metadata={}, page_content='"Keep airtight until you wish to serve these.", "Otherwise, enjoy munching on these side-by-side with a cup of hot tea!"]\\nIngridents: ["condensed milk", "bread", "coconut", "butter"]\\nlink: www.food.com/recipe/tried-and-true-recipe-for-crispy-and-tasty-snack-coconut-fingers-47569\', \'title: Chocolate Chip Kiss Cookies\\ndirections: ["Remove foil from Kisses.", "In a large bowl, combine butter, sugars and vanilla. Beat well. Add flour; blend until smooth. Stir in chips. Mold scant tablespoon of dough around each Kiss, covering completely.", "Shape into ball and place on ungreased cookie sheet.'),
 Document(metadata={}, page_content='"eggs", "lemon extract", "pecans", "raisins", "flour", "mayonnaise", "candied pineapple", "candied cherries"]\\nlink: www.cookbooks.com/Recipe-Details.aspx?id=237790\', \'title: Rice Pudding\\ndirections: ["Scald milk.", "Add sugar, salt and margarine.", "Stir until margarine is melted.", "Slowly add milk mixture to e

In [76]:
llm = ChatNVIDIA(model="meta/llama3-70b-instruct")

In [77]:
prompt = PromptTemplate(
    template="""
      You are a helpful assistant.
      Answer ONLY from the provided context.
      Context is about food recipe which includes title, direction, ingridients and link to the website of that food recipe.
      title is the name of the food recipe.
      direction describes the steps or process of making that food.
      ingridents describes that what are the things required to make that food.

      user will provide only ingridents that they have and you need to give them top 3 or 5 food recipe from the context relevant to the user ingridents.
      In your response you will provide title of that recipe, ingridents to make that food, how to make it (direction).
      the direction you provide has to be step wise and named is as steps not direction.

      If the context is insufficient, just say you don't know.

      {context}
      Ingriidents: {ingridents}
    """,
    input_variables = ['context', 'ingridents']
)

In [78]:
ingridents         = "vanilla,brown sugar,milk,butter,cream"
retrieved_docs    = retriever.invoke(ingridents)

In [79]:
retrieved_docs

[Document(metadata={}, page_content='"In a medium-sized mixing bowl, cream the butter until light and smooth.", "Add the sugars and cream together with the butter until fully incorporated.", "Add the egg and beat rapidly until smooth.", "Mix in the vanilla extract, then add the flour, baking soda, and crushed red pepper flakes. (You are supposed to sift the flour and baking soda together before adding them into the mixture, but I don\\\'t.)", "Lastly, add the milk chocolate morsels."]\\nIngridents: ["egg", "vanilla", "white granulated sugar", "brown sugar", "baking soda", "milk chocolate", "red pepper", "butter"]\\nlink:'),
 Document(metadata={}, page_content='minutes until brown.", "Drain on towel.", "Yields 30 appetizers."]\\nIngridents: ["oil", "cheddar cheese", "refrigerator biscuits"]\\nlink: www.cookbooks.com/Recipe-Details.aspx?id=596893\', \'title: Vanilla Ice Cream\\ndirections: ["In a double boiler or large saucepan, combine sugar and flour. Add eggs and 1/2 quart milk.", "Co

In [80]:
context_text = "\n\n".join(doc.page_content for doc in retrieved_docs)
context_text

'"In a medium-sized mixing bowl, cream the butter until light and smooth.", "Add the sugars and cream together with the butter until fully incorporated.", "Add the egg and beat rapidly until smooth.", "Mix in the vanilla extract, then add the flour, baking soda, and crushed red pepper flakes. (You are supposed to sift the flour and baking soda together before adding them into the mixture, but I don\\\'t.)", "Lastly, add the milk chocolate morsels."]\\nIngridents: ["egg", "vanilla", "white granulated sugar", "brown sugar", "baking soda", "milk chocolate", "red pepper", "butter"]\\nlink:\n\nminutes until brown.", "Drain on towel.", "Yields 30 appetizers."]\\nIngridents: ["oil", "cheddar cheese", "refrigerator biscuits"]\\nlink: www.cookbooks.com/Recipe-Details.aspx?id=596893\', \'title: Vanilla Ice Cream\\ndirections: ["In a double boiler or large saucepan, combine sugar and flour. Add eggs and 1/2 quart milk.", "Cook over medium heat, stirring constantly until mixture coats spoon.", "Re

In [81]:
final_prompt = prompt.invoke({"context": context_text, "ingridents":ingridents})

In [82]:
final_prompt

StringPromptValue(text='\n      You are a helpful assistant.\n      Answer ONLY from the provided context.\n      Context is about food recipe which includes title, direction, ingridients and link to the website of that food recipe.\n      title is the name of the food recipe.\n      direction describes the steps or process of making that food.\n      ingridents describes that what are the things required to make that food.\n\n      user will provide only ingridents that they have and you need to give them top 3 or 5 food recipe from the context relevant to the user ingridents.\n      In your response you will provide title of that recipe, ingridents to make that food, how to make it (direction).\n      the direction you provide has to be step wise and named is as steps not direction.\n\n      If the context is insufficient, just say you don\'t know.\n\n      "In a medium-sized mixing bowl, cream the butter until light and smooth.", "Add the sugars and cream together with the butter un

In [83]:
answer = llm.invoke(final_prompt)
print(answer.content)

Here are the top 3 recipes that match the ingredients you have:

**Recipe 1: Butter Pecan Ice Cream**
Title: Butter Pecan Ice Cream
Ingridents: ["butter", "brown sugar", "vanilla", "milk", "cream"]
Steps:
1. Melt the butter in a small saucepan.
2. Over low heat, cook the pecans until they are lightly browned, about 3 minutes.
3. Place on a paper towel to drain and cool.
4. In an 8-cup measuring cup with a spout, combine the remaining ingredients.
5. Whisk until the sugar is dissolved.
6. Pour the mixture into an ice cream maker and freeze according to manufacturers' instructions.

**Recipe 2: Vanilla Ice Cream**
Title: Vanilla Ice Cream
Ingridents: ["vanilla", "sugar", "eggs", "flour", "milk", "cream"]
Steps:
1. In a double boiler or large saucepan, combine sugar and flour. Add eggs and 1/2 quart milk.
2. Cook over medium heat, stirring constantly until mixture coats spoon.
3. Remove from heat, cool slightly; add vanilla and enough milk to fill freezer can.
4. Freeze according to direc